In [ ]:
import matplotlib.pyplot as plt
import cv2
import glob
import os
import shutil
from cv2 import dnn_superres
import ipywidgets as widgets
from IPython.display import display
import random
from IPython.display import clear_output

## see the image files

In [ ]:
dir_path = 'original_images'

files = os.listdir(dir_path)
print(len(files))
files[:5]

In [ ]:
# count the original image number

num_files = len(files)
num_files

In [ ]:
# make list of zeros and ones for each image in random order

random_list = [random.randint(0, 1) for i in range(num_files)]
print(*random_list)

## first image check : color changes, but corrected in the following code

In [ ]:
# load image

img_o = cv2.imread(dir_path + '/' + files[0])[:,:,::-1]
img   = cv2.cvtColor(img_o, cv2.COLOR_BGR2RGB) 
plt.imshow(img)

In [ ]:
# get image size

height_o, width_o, channels = img.shape[:3]
print( width_o, height_o)

In [ ]:
# make directory for images made

os.makedirs('resized_images', exist_ok=True)

## first, determine duration of 1 image animation

In [ ]:
one_animation_duration = input('impot duration for one image in seconds :  ')

In [ ]:
one_animation_image_num = int(float(one_animation_duration) * 15)          # 10 = flames / sec
print('number of images =  ', one_animation_image_num, '  pcs')

In [ ]:
zoom_rate = 0.8

In [ ]:
# crop test : reduce pixels from 4 sides

pixel_to_reduce_in_x = zoom_rate * one_animation_image_num
print('pixel_to_reduce_in_x = ', pixel_to_reduce_in_x)
pixel_to_reduce_h = int(pixel_to_reduce_in_x)
pixel_to_reduce_w = int(pixel_to_reduce_in_x * (2/3))

img_resize  = img[pixel_to_reduce_h : height_o - pixel_to_reduce_w, pixel_to_reduce_w : width_o - pixel_to_reduce_w]

height_rsz, width_rsz, channels = img_resize.shape[:3]
print('width, height =  ', width_rsz, height_rsz)
plt.imshow(img_resize)

In [ ]:
# zoom in function

def zoom_in(img, pixcel_reduce, animation_image_order):        # min pixcel_reduce = around 3 depends on the original size
    height_o, width_o, channels = img.shape[:3]
    
    pixel_to_reduce_in_x = pixcel_reduce * animation_image_order
    pixel_to_reduce_h = int(pixel_to_reduce_in_x)
    pixel_to_reduce_w = int(pixel_to_reduce_in_x * (2/3))
    
    img_resize  = img[pixel_to_reduce_h : height_o - pixel_to_reduce_w, pixel_to_reduce_w : width_o - pixel_to_reduce_w]
    img2 = cv2.resize(img_resize, (width_rsz, height_rsz))
    
    return img2 

In [ ]:
# zoom out function

def zoom_out(img, pixcel_reduce, animation_image_order):        # min pixcel_reduce = around 3 depends on the original size
    height_o, width_o, channels = img.shape[:3]
    
    pixel_to_reduce_in_x = pixcel_reduce * (one_animation_image_num - animation_image_order)
    pixel_to_reduce_h = int(pixel_to_reduce_in_x)
    pixel_to_reduce_w = int(pixel_to_reduce_in_x * (2/3))
    
    img_resize  = img[pixel_to_reduce_h : height_o - pixel_to_reduce_w, pixel_to_reduce_w : width_o - pixel_to_reduce_w]
    img2 = cv2.resize(img_resize, (width_rsz, height_rsz))
    
    return img2 

In [ ]:
# make images for animation using all images

image_global_num = 0

for i in range(num_files):
   
    if random_list[i] == 0:
        original_image = files[i]
        img_o = cv2.imread(dir_path + '/' + original_image)[:,:,::-1]
        img   = cv2.cvtColor(img_o, cv2.COLOR_BGR2RGB)
        for operation_num in range(one_animation_image_num):
            image_continuous_num = str(image_global_num).zfill(5)
            if operation_num <= one_animation_image_num:
                img_resize = zoom_in(img, zoom_rate, operation_num)
            else:
                img_resize = zoom_in(img, zoom_rate, one_animation_image_num) 
            cv2.imwrite('resized_images/' + image_continuous_num + '_resized_image.png' , img_resize)
            image_global_num += 1

    if random_list[i] == 1:
        original_image = files[i]
        img_o = cv2.imread(dir_path + '/' + original_image)[:,:,::-1]
        img   = cv2.cvtColor(img_o, cv2.COLOR_BGR2RGB)
        for operation_num in range(one_animation_image_num):
            image_continuous_num = str(image_global_num).zfill(5)
            if operation_num <= one_animation_image_num:
                img_resize = zoom_out(img, zoom_rate, operation_num)
            else:
                img_resize = zoom_out(img, zoom_rate, one_animation_image_num)        
            cv2.imwrite('resized_images/' + image_continuous_num + '_resized_image.png' , img_resize)        
            image_global_num += 1    
    
    clear_output(True)
    print('processing in : ', i + 1, ' out of ', num_files)
            
print('Done!')

## make animation and save individually

## animation head name
zoom_in_out_animation_

In [ ]:
name_of_animation_head = 'zoom_in_out_animation_'

In [ ]:
# read PNG files in the target directory and make their list

def make_img_array(counter, one_animation_image_num):
    
    filenames = glob.glob("resized_images/*.png")
    img_array = []
    
    for filename_counter in range(counter * one_animation_image_num, (counter + 1) * one_animation_image_num):
        filename = filenames[filename_counter]
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)
        
    return img_array, size

In [ ]:
for counter in range(len(files)):
    count_tail = str(counter).zfill(3)
    img_array, size = make_img_array(counter, one_animation_image_num)
    #print(len(img_array), count_tail)

In [ ]:
# Change images to mp4 animation

def make_mp4_animation(name, img_array):
    out = cv2.VideoWriter(name + '.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 15, size)

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [ ]:
for counter in range(len(files)):
    count_tail = str(counter).zfill(3)
    name = name_of_animation_head + count_tail
    img_array, size = make_img_array(counter, one_animation_image_num)
    make_mp4_animation(name, img_array)

In [ ]:
print('\nJust suspended for safety.\n')
button = widgets.Button(description = 'CONTUNUE?')
display(button)

## move the file to add music if necessary

In [ ]:
path_to_move = 'D:/+Python_code_picture_related/+make_mp4_animation'

In [ ]:
new_path = shutil.move(name + '.mp4', path_to_move)

## move no use directries

In [ ]:
# make new directry to move used files

os.makedirs('x_folders_to_remove', exist_ok=True)

In [ ]:
current_path = 'resized_images'
not_use_path = 'x_folders_to_remove'

new_path = shutil.move(current_path, not_use_path)

 ## delete no use folders

In [ ]:
shutil.rmtree('x_folders_to_remove')